In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import healpy as hp

from qubic.lib.MapMaking.Qatmosphere_2d import AtmosphereMaps

import sys
import yaml

import qubic
from qubic.lib.Instrument.Qacquisition import QubicDualBand

from qubic.lib.MapMaking.Qmaps import InputMaps
from qubic.lib.MapMaking.Qcg import PCGAlgorithm
from pyoperators.iterative.core import AbnormalStopIteration

from pyoperators import *



comm = MPI.COMM_WORLD
rank = comm.Get_rank()

%matplotlib inline

In [ ]:
# Import simulation parameters
with open('params.yml', 'r') as file:
    params = yaml.safe_load(file) 

In [ ]:
# Call the class which build the atmosphere maps
atm = AtmosphereMaps(params)

In [ ]:
# Import the atm absorption spectrum
abs_spectrum = atm.absorption_spectrum()

plt.plot(abs_spectrum)
plt.ylim(0, 0.0002)

In [ ]:
# Import the atm temperature maps
atm_maps = atm.get_temp_maps(atm.rho_map)
print(atm_maps.shape)

# Import the atm integrated absorption spectrum
integrated_abs_spectrum, frequencies = atm.integrated_absorption_spectrum()

In [ ]:
mean_atm_maps = []
for i in range(atm_maps.shape[0]):
    mean_atm_maps.append(np.mean(atm_maps[i, :]))
plt.figure()
plt.plot(frequencies, mean_atm_maps, '.')
plt.title('Atmosphere maps spectrum')
plt.xlabel('Frequency (GHz)')
plt.ylabel(r'Mean temperature ($\mu K_{CMB}$)')
plt.figure()
plt.plot(frequencies, integrated_abs_spectrum, '.')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Integrated absorption spectrum')
plt.title('Integrated absorption spectrum')

In [ ]:
plt.imshow(atm_maps[0], cmap='jet')
plt.colorbar(label='µK_CMB')
plt.title('Atmosphere temperature map at {:.2f} GHz'.format(frequencies[0]))

In [ ]:
# Build input maps : CMB + Atmsophere (only for I map)
input_maps = np.zeros((len(frequencies), hp.nside2npix(params['nside']), 3))
input_maps[:, :, 0] += atm.get_healpy_atm_maps_2d(atm_maps)
print(input_maps.shape)

for irec in range(input_maps.shape[0]):
    index = np.where(input_maps[irec, :, 0] != 0)
    input_maps[irec, index, 0] -= np.mean(input_maps[irec, index, 0])

hp.mollview(input_maps[-1, :, 0] ,cmap='jet', unit='µK_CMB', norm='hist', title='Input map {:.2f} GHz'.format(frequencies[-1]))
hp.graticule()

In [ ]:
# Compute the Nrec maps from the Nsub maps
true_maps = np.zeros((params['nrec'], 12*params['nside']**2, 3))
true_frequencies = np.zeros(params['nrec'])
fsub = int(params['nsub_in'] / params['nrec'])
for i in range(params['nrec']):
    true_maps[i, :, :] = np.mean(input_maps[i*fsub:(i+1)*fsub, :, :], axis=0)
    true_frequencies[i] = np.mean(frequencies[i*fsub:(i+1)*fsub])
print(true_maps.shape) 
print(true_frequencies)   
plt.plot(true_frequencies, np.mean(true_maps, axis=1)[..., 0], '.')
plt.ylabel(r'Mean temperature ($\mu K_{CMB}$)')
plt.xlabel('Frequency (GHz)')
plt.xlim(130, 250)

In [ ]:
for inu in range(len(true_frequencies)):
    hp.mollview(true_maps[inu, :, 0], cmap='jet', title='Input - {:.2f} GHz'.format(true_frequencies[inu]))

# Map-making

In [ ]:
# Build the QUBIC operators
H_tod_qubic = QubicDualBand(atm.qubic_dict, nsub=params['nsub_in'], nrec=params['nsub_in']).get_operator()
Qacq = QubicDualBand(atm.qubic_dict, nsub=params['nsub_in'], nrec=params['nrec'])
if params['nrec']==2:
    H_rec_qubic = Qacq.get_operator().operands[0]
else:
    H_rec_qubic = Qacq.get_operator()
invN_qubic = Qacq.get_invntt_operator()
tod_qubic = H_tod_qubic(input_maps).ravel()

In [ ]:
coverage = Qacq.coverage

covnorm = coverage / coverage.max()
seenpix = covnorm > params['coverage_cut']

center = np.array([0, -57])
qubic_patch = qubic.lib.Qsamplings.equ2gal(center[0], center[1])

In [ ]:
print('H_rec', H_rec_qubic.shapein, H_rec_qubic.shapeout)
print('invN', invN_qubic.shapein, invN_qubic.shapeout)
print('H_tod', H_tod_qubic.shapein, H_tod_qubic.shapeout)

In [ ]:
H_tod = H_tod_qubic
R = ReshapeOperator(H_rec_qubic.shapeout, invN_qubic.shapein)

H_rec = R*H_rec_qubic
invN = invN_qubic
tod = tod_qubic

In [ ]:
print('H_rec_qubic', H_rec_qubic.shapein, H_rec_qubic.shapeout)
print("H_rec", H_rec.shapein, H_rec.shapeout)
print("invN", invN.shapein, invN.shapeout)
print('invN qubic', invN_qubic.shapein, invN_qubic.shapeout)
print("TOD", tod.shape)

In [ ]:
# Build PCG
R = ReshapeOperator(tod.shape, invN_qubic.shapein)
A = H_rec.T * invN * H_rec
b = H_rec.T * invN * R(tod)
x0 = true_maps*0

In [ ]:
print("Number of Data Sets :", len(H_rec_qubic.operands)) # operands[0] = QUBIC / operands[1] = Planck
print("Number of Sub-bands within each Frequency Bands :", len(H_rec_qubic.operands[0].operands))
print("Number of Frequency Bands :", len(H_rec_qubic.operands[0].operands[0].operands)) # operands[0] = 150 GHz / operands[1] = 220 GHz
print("Acquisition Operator for the first Sub-Bands of QUBIC :")
print(H_rec_qubic.operands[0].operands[0].operands[0].operands)

In [ ]:
print(len(H_rec_qubic.operands[0].operands[0].operands))

In [ ]:
# Build Preconditionner
stacked_dptdp_inv = np.zeros((params['nrec'],12*params['nside']**2))
no_det = 992

### Loop on Frequency Sub-Bands within each band
for i_nsub in range(int(params["nrec"]/2)):
    ### Loop on Frequency Bands
    for j_fp in range(2):
        ### Extract Operators
        H_single = H_rec_qubic.operands[0].operands[i_nsub].operands[j_fp]
        D = H_single.operands[1]
        P = H_single.operands[-1]
        sh = P.matrix.data.index.shape

        ### Compute the map P^t P
        point_per_det = int(sh[0] / no_det)
        mapPtP_perdet_seq = np.zeros((no_det, 12 * params['nside']**2))
        sample_ranges = [(det * point_per_det, (det + 1) * point_per_det) for det in range(no_det)]
        for det, (start, end) in enumerate(sample_ranges):
            indices = P.matrix.data.index[start:end, :]  
            weights = P.matrix.data.r11[start:end, :]
            flat_indices = indices.ravel()
            flat_weights = weights.ravel()

            mapPitPi = np.zeros(12 * params['nside']**2)
            np.add.at(mapPitPi, flat_indices, flat_weights**2)

            mapPtP_perdet_seq[det, :] = mapPitPi
            
        D_elements = D.data
        D_sq = D_elements**2
        mapPtP_seq_scaled = D_sq[:, np.newaxis] * mapPtP_perdet_seq 
        dptdp = mapPtP_seq_scaled.sum(axis = 0)
        dptdp_inv = 1 / dptdp
        dptdp_inv[np.isinf(dptdp_inv)] = 0.
        stacked_dptdp_inv[int(i_nsub*(int(params['nrec']/2)) + j_fp)] = dptdp_inv

M = BlockDiagonalOperator( \
                    [DiagonalOperator(ci, broadcast='rightward') for ci in stacked_dptdp_inv],
                    new_axisin=0)
        

In [ ]:
stacked_dptdp_inv.shape

In [ ]:
# Run PCG
algo = PCGAlgorithm(
    A,
    b,
    comm,
    x0=x0,
    tol=1e-10,
    maxiter=100,
    disp=True,
    M=M,
    center=[0, -57],
    reso=15,
    seenpix=seenpix,
    input=true_maps,
)
try:
    output = algo.run()
    success = True
    message = 'Success'
except AbnormalStopIteration as e:
    output = algo.finalize()
    success = False
    message = str(e)

In [ ]:
plt.plot(output['convergence'])
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Convergence')

In [ ]:

plt.figure(figsize=(12, 12), dpi=200)
k=1
true_maps[:, ~seenpix, :] = hp.UNSEEN
output['x'][:, ~seenpix, :] = hp.UNSEEN

stk = ['I', 'Q', 'U']
istk = 0
n_sig = 3
reso = 15

for inu in range(output['x'].shape[0]):
    sigma = np.std(true_maps[inu, seenpix, istk])
    hp.gnomview(true_maps[inu, :, istk], min=np.min(true_maps[inu, seenpix, istk]), max=np.max(true_maps[inu, seenpix, istk]), cmap='jet', rot=qubic_patch,title='{} - Input - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=reso, sub=(output['x'].shape[0], 3, k), notext=True)
    hp.gnomview(output['x'][inu, :, istk], min=np.min(true_maps[inu, seenpix, istk]), max=np.max(true_maps[inu, seenpix, istk]), cmap='jet', rot=qubic_patch,title='{} - Output - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=reso, sub=(output['x'].shape[0], 3, k+1), notext=True)
    hp.gnomview(output['x'][inu, :, istk] - true_maps[inu, :, 0], cmap='jet', rot=qubic_patch,title='{} - Residual - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=reso, sub=(output['x'].shape[0], 3, k+2), notext=True)
    k+=3

In [ ]:
for inu in range(output['x'].shape[0]):
    sigma = np.std(true_maps[inu, seenpix, istk])
    hp.mollview(output['x'][inu, :, istk] - true_maps[inu, :, istk], cmap='jet',title='{} - Residual - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]))
    hp.graticule()

In [ ]:
plt.figure(figsize=(12, 12), dpi=200)

k=1
true_maps[:, ~seenpix, :] = hp.UNSEEN
output['x'][:, ~seenpix, :] = hp.UNSEEN

istk = 1
n_sig = 2

for inu in range(output['x'].shape[0]):
    sigma = np.std(true_maps[inu, seenpix, istk])
    hp.gnomview(true_maps[inu, :, istk], cmap='jet', rot=qubic_patch,title='{} - Input - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=15, sub=(output['x'].shape[0], 3, k), notext=True)
    hp.gnomview(output['x'][inu, :, istk], cmap='jet', rot=qubic_patch,title='{} - Output - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=15, sub=(output['x'].shape[0], 3, k+1), notext=True)
    hp.gnomview(output['x'][inu, :, istk] - true_maps[inu, :, istk], cmap='jet', rot=qubic_patch,title='{} - Residual - {:.2f} GHz'.format(stk[istk], true_frequencies[inu]), reso=15, sub=(output['x'].shape[0], 3, k+2), notext=True)
    k+=3